In [32]:
import requests
import pandas as pd
from datetime import datetime

def get_weekly_average_temperatures(cities):
    # Créer une liste pour stocker les données
    data = []

    # Mappage des stations pour chaque ville
    city_stations = {
        "Paris": "ORLY",
        "Marseille": "MARSEILLE-PROVENCE",
        "Lyon": "LYON-ST EXUPERY",
        "Bordeaux": "BORDEAUX-MERIGNAC",
        "Lille": "LILLE-LESQUIN"
    }

    # Boucler à travers les villes
    for city in cities:
        station = city_stations[city]

        # Endpoint de l'API
        URL = "https://public.opendatasoft.com/api/records/1.0/search/"
        ressource = "?dataset=donnees-synop-essentielles-omm&q="

        # Créer l'URL de l'API pour l'année 2022
        start_date = "2008-01-01"
        end_date = datetime.today().strftime('%Y-%m-%d')
        date_fork = f"date%3A%5B{start_date}+TO+{end_date}%5D"
        station_filter = f"&refine.nom={station}"
        row_limit = "&rows=10000"
        endpoint = URL + ressource + date_fork + station_filter + row_limit

        # Effectuer la requête GET à l'API
        response = requests.get(endpoint)

        # Calculer la température moyenne par semaine
        weekly_temperatures = {}
        for record in response.json()["records"]:
            date = record["fields"]["date"][:10]  # Extraire la date du format YYYY-MM-DD HH:MM:SS
            temperature = record["fields"].get("tc")
            if not temperature:
                continue  # passer à l'itération suivante si la clé "tc" n'est pas présente

            week_start = pd.to_datetime(date).to_period('W-TUE').start_time.strftime('%Y-%m-%d')
            if week_start in weekly_temperatures:
                weekly_temperatures[week_start].append(temperature)
            else:
                weekly_temperatures[week_start] = [temperature]

        # Calculer la température moyenne pour chaque semaine et ajouter les données à la liste
        for week_start, temperatures in weekly_temperatures.items():
            temperature_mean = sum(temperatures) / len(temperatures)
            data.append({
                "City": city,
                "Week Number": pd.to_datetime(week_start).week,
                "Start Date": week_start,
                "End Date": (pd.to_datetime(week_start) + pd.DateOffset(days=6)).strftime('%Y-%m-%d'),
                "Average Temperature": round(temperature_mean, 2)
            })

    # Créer un DataFrame à partir des données
    df = pd.DataFrame(data)

    # Utiliser la fonction pivot_table pour avoir une colonne par ville et la semaine du mercredi au mercredi
    df_pivot = df.pivot_table(index=["Week Number", "Start Date", "End Date"], columns="City", values="Average Temperature", aggfunc='first')

    # Réinitialiser l'index pour obtenir une DataFrame plat
    df_pivot = df_pivot.reset_index()
    
    df_pivot["Moyenne"] = round(df_pivot.mean(axis=1), 2)

    return df_pivot

# Liste des villes à considérer
cities = ["Paris", "Marseille", "Lyon", "Bordeaux", "Lille"]

# Appel de la fonction pour obtenir les températures moyennes par semaine pour 2022
weekly_average_temperatures_df = get_weekly_average_temperatures(cities)

# Afficher le DataFrame
print(weekly_average_temperatures_df)


# Enregistrer le DataFrame au format CSV
weekly_average_temperatures_df.to_csv("weekly_average_temperatures.csv", index=False)

City  Week Number  Start Date    End Date  Bordeaux  Lille  Lyon  Paris  \
0               1  2010-01-06  2010-01-12     -1.98  -2.09 -2.41  -3.15   
1               1  2011-01-05  2011-01-11      8.57   4.19  7.44   7.21   
2               1  2012-01-04  2012-01-10      9.10   8.01  4.18   8.73   
3               1  2013-01-02  2013-01-08      7.07   7.43  4.77   6.24   
4               1  2014-01-01  2014-01-07     11.27   9.93  9.92   9.23   
..            ...         ...         ...       ...    ...   ...    ...   
703            52  2021-12-29  2022-01-04     12.88  11.33  9.01  10.57   
704            52  2022-12-28  2023-01-03     12.42   9.80  9.95  10.89   
705            53  2009-12-30  2010-01-05      3.62  -2.61 -0.34  -1.73   
706            53  2015-12-30  2016-01-05     10.43   6.72  7.29   8.30   
707            53  2020-12-30  2021-01-05      2.50   2.22  1.69   2.75   

City  Moyenne  
0       -1.73  
1        5.68  
2        6.20  
3        5.30  
4        8.27  
.. 

/tmp/ipykernel_863019/964862380.py:71: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_pivot["Moyenne"] = round(df_pivot.mean(axis=1), 2)
